In [ ]:
import requests
import json

In [ ]:
def load_communes():
    try:
        response = requests.get("https://geo.api.gouv.fr/communes")
        if response.status_code != 200:
            raise Exception(f"Error when querying geo.api.gouv.fr: {response.status_code}")
        data = json.loads(response.content)
        return data
    except Exception as e:
        raise e

In [ ]:
communes = load_communes()

In [ ]:
departements = sorted(list(set(c["codeDepartement"] for c in communes)))

In [ ]:
def load_dept_geojson(codeDept):
    try:
        resp = requests.get(f"https://geo.api.gouv.fr/departements/{codeDept}/communes?format=geojson&geometry=contour")
        if not resp.status_code == 200:
            raise Exception(f"Error when querying geo.api.gouv.fr: {resp.status_code}")
        data = json.loads(resp.content)
        return data
    except Exception as e:
        raise e

In [ ]:
communes_geojson = [load_dept_geojson(d) for d in departements]

In [ ]:
communes_geojson[3]

In [ ]:
# Aggregate in one GeoJSON

all_features = [feature for c in communes_geojson for feature in c["features"]]

In [ ]:
all_features[:4]

In [ ]:

all_communes_geojson = {
    "type": 'FeatureCollection',
    "features": all_features
}

# Display polygons on the map

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[48.85341,2.3488], zoom_start=7)

In [ ]:
# TODO better graphics (different color per commune, finer line between polygons, tooltip when hovering over commune?)

folium.GeoJson(all_communes_geojson).add_to(m)

In [ ]:
# WARNING this will require lots of memory

m